In [25]:
pip install opencv-python numpy ultralytics gdown

In [26]:
!apt-get update -y
!apt-get install -y ffmpeg libsm6 libxext6 unrar


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (2,662 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsm6 is already the newest version (2:1.2.3-1bui

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [28]:
!ffmpeg -y \
-i /content/drive/MyDrive/ADAS_Dataset/Test/9.mp4 \
-vcodec libx264 -pix_fmt yuv420p \
-acodec aac \
/content/drive/MyDrive/ADAS_Dataset/Test/9_fixed.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [36]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

VIDEO_IN = "/content/drive/MyDrive/ADAS_Dataset/Test/9.mp4"
VIDEO_OUT = "output_adas.mp4"

ROI_POLY_NORM = np.array([
    [0.15, 1.00],
    [0.85, 1.00],
    [0.60, 0.55],
    [0.40, 0.55],
], dtype=np.float32)


LANE_DEVIATION_THRESH = 60

NEAR_Y_FRAC   = 0.80
BIG_BOX_FRAC  = 0.25


In [37]:
cap = cv2.VideoCapture(VIDEO_IN)
print("Opened:", cap.isOpened())
cap.release()


Opened: True


In [38]:

yolo_model = YOLO("yolov8n.pt")

VEHICLE_CLASSES = {2, 3, 5, 7}


In [39]:
def region_of_interest(edges):
    h, w = edges.shape
    mask = np.zeros_like(edges)

    polygon = np.array([[
        (0,   h),
        (w,   h),
        (int(0.55*w), int(0.6*h)),
        (int(0.45*w), int(0.6*h)),
    ]], dtype=np.int32)

    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(edges, mask)








In [43]:
def detect_lane_lines(frame):
    h, w = frame.shape[:2]

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    edges = cv2.Canny(blur, 50, 150)

    cropped = region_of_interest(edges)

    lines = cv2.HoughLinesP(
        cropped,
        rho=1,
        theta=np.pi / 180,
        threshold=50,
        minLineLength=40,
        maxLineGap=120,
    )

    if lines is None:
        return None, None, w // 2

    left_lines, right_lines = [], []

    for x1, y1, x2, y2 in lines[:, 0]:
        if x1 == x2:
            continue

        slope = (y2 - y1) / (x2 - x1)
        if abs(slope) < 0.5:
            continue

        if slope < 0 and x1 < w / 2 and x2 < w / 2:
            left_lines.append((x1, y1, x2, y2))
        elif slope > 0 and x1 > w / 2 and x2 > w / 2:
            right_lines.append((x1, y1, x2, y2))

    def avg_line(lines):
        if not lines:
            return None
        lines = np.array(lines)
        x1, y1, x2, y2 = lines.mean(axis=0).astype(int)
        return (x1, y1), (x2, y2)

    left = avg_line(left_lines)
    right = avg_line(right_lines)

    if left and right:
        lx2, ly2 = left[1]
        rx2, ry2 = right[1]
        lane_center_x = (lx2 + rx2) // 2
    else:
        lane_center_x = w // 2

    return left, right, lane_center_x


In [44]:
def draw_lane_overlay(frame, left, right, lane_center_x):
    h, w = frame.shape[:2]
    overlay = frame.copy()

    if left:
        cv2.line(overlay, left[0], left[1], (0, 255, 0), 8)
    if right:
        cv2.line(overlay, right[0], right[1], (0, 255, 0), 8)

    if left and right:
        pts = np.array([left[0], left[1], right[1], right[0]], np.int32)
        cv2.fillConvexPoly(overlay, pts, (0, 255, 0))
        alpha = 0.3
        frame = cv2.addWeighted(overlay, alpha, frame, 1-alpha, 0)

    cv2.line(frame, (lane_center_x, h), (lane_center_x, int(0.6*h)), (255, 255, 0), 2)

    cv2.line(frame, (w//2, h), (w//2, int(0.6*h)), (255, 255, 255), 1)

    return frame


In [45]:
next_id = 0
tracks = {}
track_in_roi = {}
total_vehicles_entered = 0


In [46]:
def get_roi_mask(shape):
    h, w = shape[:2]
    poly = ROI_POLY_NORM.copy()
    poly[:, 0] *= w
    poly[:, 1] *= h
    poly = poly.astype(np.int32)
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [poly], 255)
    return mask, poly

def box_in_roi(box, roi_mask):
    x1, y1, x2, y2 = map(int, box)
    cx = (x1 + x2) // 2
    cy = (y1 + y2) // 2
    h, w = roi_mask.shape
    if cx < 0 or cx >= w or cy < 0 or cy >= h:
        return False
    return roi_mask[cy, cx] > 0


next_id = 0
tracks = {}
counted_ids = set()
total_vehicles_entered = 0

def update_tracks(detections, roi_mask):
    """
    detections: list of (box, in_roi_bool)
    """
    global next_id, tracks, track_in_roi, total_vehicles_entered

    new_tracks = {}
    new_in_roi = {}
    used_old_ids = set()

    for box, in_roi in detections:
        x1, y1, x2, y2 = map(int, box)
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2


        assigned_id = None
        min_dist = 60

        for tid, (px, py) in tracks.items():
            if tid in used_old_ids:
                continue
            dist = np.hypot(cx - px, cy - py)
            if dist < min_dist:
                min_dist = dist
                assigned_id = tid


        if assigned_id is None:
            assigned_id = next_id
            next_id += 1
            track_in_roi.setdefault(assigned_id, False)

        new_tracks[assigned_id] = (cx, cy)
        used_old_ids.add(assigned_id)

        was_in_roi = track_in_roi.get(assigned_id, False)
        if in_roi and not was_in_roi:
            total_vehicles_entered += 1

        new_in_roi[assigned_id] = was_in_roi or in_roi

    tracks = new_tracks
    track_in_roi = new_in_roi



In [47]:
def assess_collision_risk(box, h):
    x1, y1, x2, y2 = map(int, box)
    box_h = y2 - y1
    bottom_y = y2

    near_bottom = bottom_y > NEAR_Y_FRAC * h
    big_box    = box_h > BIG_BOX_FRAC * h

    return near_bottom or big_box


def lane_deviation_warning(frame, lane_center_x):
    h, w = frame.shape[:2]
    center_x = w // 2
    dev = lane_center_x - center_x

    msg = None
    if abs(dev) > LANE_DEVIATION_THRESH:
        direction = "RIGHT" if dev > 0 else "LEFT"
        msg = f"LANE DEPARTURE {direction}"

    return msg, dev


In [48]:
tracker_state = None


In [49]:
cap = cv2.VideoCapture(VIDEO_IN)
if not cap.isOpened():
    raise RuntimeError("Could not open input video")

fps = cap.get(cv2.CAP_PROP_FPS) or 25
w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out    = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (w, h))

roi_mask, roi_poly = get_roi_mask((h, w))

frame_idx = 0
seen_ids_in_roi = set()
total_vehicles_entered = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    left, right, lane_center_x = detect_lane_lines(frame)
    frame = draw_lane_overlay(frame, left, right, lane_center_x)

    yolo_results = yolo_model.track(
        frame,
        imgsz=640,
        conf=0.4,
        iou=0.5,
        persist=True,
        verbose=False
    )[0]

    detections = []

    boxes = yolo_results.boxes
    if boxes.id is not None:
        ids = boxes.id.cpu().numpy().astype(int)
    else:
        ids = np.array([-1] * len(boxes))

    for box, cls, tid in zip(boxes.xyxy, boxes.cls, ids):
        cls_id = int(cls.item())
        if cls_id not in VEHICLE_CLASSES:
            continue
        box = box.cpu().numpy()
        in_roi = box_in_roi(box, roi_mask)
        detections.append((tid, box, in_roi))

        x1, y1, x2, y2 = map(int, box)
        color = (0, 255, 255) if in_roi else (0, 255, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        if tid != -1:
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            cv2.putText(frame, str(tid), (cx, cy),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 0, 255), 1, cv2.LINE_AA)

    for tid, box, in_roi in detections:
        if tid == -1:
            continue
        if in_roi and tid not in seen_ids_in_roi:
            seen_ids_in_roi.add(tid)
            total_vehicles_entered += 1

    cv2.polylines(frame, [roi_poly], isClosed=True, color=(255, 0, 0), thickness=2)
    roi_overlay = frame.copy()
    cv2.fillPoly(roi_overlay, [roi_poly], (255, 0, 0))
    frame = cv2.addWeighted(roi_overlay, 0.15, frame, 0.85, 0)

    danger = False
    for _, box, in_roi in detections:
        if not in_roi:
            continue
        if assess_collision_risk(box, h):
            danger = True
            break

    if danger:
        cv2.putText(frame, "FORWARD COLLISION RISK!",
                    (40, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (0, 0, 255), 3, cv2.LINE_AA)

    lane_msg, dev = lane_deviation_warning(frame, lane_center_x)
    if lane_msg:
        cv2.putText(frame, lane_msg, (40, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (0, 165, 255), 2, cv2.LINE_AA)

    cv2.putText(frame, f"Vehicles in ROI: {total_vehicles_entered}",
                (40, h-40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                (255, 255, 255), 2, cv2.LINE_AA)

    out.write(frame)

cap.release()
out.release()
print("Saved:", VIDEO_OUT)


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 263ms
Prepared 1 package in 79ms
Installed 1 package in 2ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

Saved: output_adas.mp4


In [ ]:
from IPython.display import Video
Video(VIDEO_OUT, embed=True)
